In [7]:
from data_processing import prepare_dataset, assign_colors
from align_func import execute_global_registration, icp_registration, colored_icp
from GA import evaluation_colored_icp, mutation, generator
from deap import algorithms, base, creator, tools
from datetime import date

import open3d as o3d
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import copy
import random
import os
import sys



In [8]:
################################ 1) Change variables here ################################ 

### File directories ###
source_position_name = "./datasets/xyz_source.xls"
source_color_name = "./datasets/sox2_source.xls"
target_position_name = "./datasets/xyz_target.xls"
target_color_name= "./datasets/sox2_target.xls"

### DEAP runs ### 

image_name = "sox2"
save_results_path = "./results"
results_name = "NSGA3_coloredICP_run"
num_deap_runs = 20


In [9]:
################################ 2) Importing files and data processing ###############################################

## Source ##
source_position_excel = [pd.read_excel(source_position_name, skiprows = [0,1,2], header = 0, usecols = [0,1,2])]
source_color_excel = [pd.read_excel(source_color_name, skiprows = [0,1,2], header = 0, usecols = [0])]

## Target ## 
target_position_excel = [pd.read_excel(target_position_name, skiprows = [0,1,2], header = 0, usecols = [0,1,2])]
target_color_excel = [pd.read_excel(target_color_name, skiprows = [0,1,2], header = 0, usecols = [0])]

positions_excel_list = source_position_excel + target_position_excel
positions_names_list = [source_position_name] + [target_position_name]
color_excel_list = source_color_excel + target_color_excel

source_color =color_excel_list[0].to_numpy(dtype='float64')
target_color =color_excel_list[1].to_numpy(dtype='float64')

source, target, source_processed, target_processed, source_fpfh, target_fpfh = prepare_dataset(positions_excel_list, positions_names_list)
assign_colors(source,target,color_excel_list,"viridis")

:: Point Cloud was not downsampled
:: Estimate normal with search radius 20.
:: Compute FPFH feature with search radius 50.
---------------------------------------
:: Point Cloud was not downsampled
:: Estimate normal with search radius 20.
:: Compute FPFH feature with search radius 50.
---------------------------------------
:: Assigned colors to point clouds


In [10]:
######################### 3) DEAP initialisations #################################

# results.fitness (MAX), results.inlier_mse (MIN), mae (MIN)
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, -1.0)) 

# A list-type individual with a fitness attribute
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Instantiate a Toolbox to register all the evolutionary operators.
toolbox = base.Toolbox()


NDIM = 10
NOBJ = 3
p = 12
toolbox.pop_size = 30
toolbox.max_gen = 500
CXPB = 1.0
MUTPB = 1.0
ref_points = tools.uniform_reference_points(NOBJ, p)


""" Registering operators""" 
toolbox.register("generator", generator)

# Structure initializers
# define 'individual' to be a single individual taking up the values generated by the toolbox.generator. So we don't 
# need to repeat the toolbox.generator function. 
# This gives us flexibilty to define each parameter with its unique distribution, instead of keeping the distribution
# the same and applying it repeatedly across each parameter.

toolbox.register("individual", tools.initIterate, creator.Individual, 
    toolbox.generator)

# define the population to be a list of individuals # We don't define n here, but in the main body to give flexibility to num individuals.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evaluation_colored_icp, source_processed = source_processed, 
                 target_processed = target_processed, source_fpfh = source_fpfh, target_fpfh = target_fpfh,
                source_color = source_color, target_color = target_color)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator 

toolbox.register("mutate", mutation, indpb = [0.2]*NDIM)

toolbox.register("select", tools.selNSGA3, ref_points = ref_points )



/home/porcupine/miniconda3/envs/open3D/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/porcupine/miniconda3/envs/open3D/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [11]:
######################### 4) DEAP main #################################


def main(iter_ = 0, run_name = "./test"):
    random.seed(iter_ )
    
    # Initialize statistics object
     ## Creating logbook for recording statistics
        
    stats_o3d_fitness = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats_o3d_rmse = tools.Statistics(lambda ind: ind.fitness.values[1])
    stats_o3d_mae = tools.Statistics(lambda ind: ind.fitness.values[2])
    mstats = tools.MultiStatistics(o3d_fitness=stats_o3d_fitness, 
                                   o3d_rmse =stats_o3d_rmse,
                                   o3d_mae = stats_o3d_mae)
    
    mstats.register("mean", lambda ind: round(sum(ind)/len(pop),3))
    mstats.register("max", lambda ind: round(np.max(ind),3))
    mstats.register("min", lambda ind: round(np.min(ind),3))
    
    logbook = tools.Logbook()
    logbook.header = ["gen", "evals", "o3d_fitness", "o3d_rmse", "o3d_mae"]
    logbook.chapters["o3d_fitness"].header = ["mean", "max"]
    logbook.chapters["o3d_rmse"].header = ["mean", "min"]
    logbook.chapters["o3d_mae"].header = ["mean", "min"]
    
    pop = toolbox.population(n= toolbox.pop_size)
        
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit[0:3]

    #Compile statistics about the population
    record = mstats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print("Initial logbook.stream", logbook.stream)
    
    # Best k individuals each gen (parameter values)
    #best_k_ind = []
    
    # Best k individuals each gen (coloredICPresults)
   # best_k_ind_results = []
    
    # Manual logging results with corr maps
    corr_map_results = []
    
    # Begin the generational process
    
    #print("Beginning generational process")
    for gen in range(1, toolbox.max_gen):
       # for ind in pop:
       #     print(f"{gen}---Parent: {ind}, Fitness: {ind.fitness.values}---")
        
        #Apply crossover and mutation to generate new offsprings. Return a list of varied individuals that are independent of their parents.
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)
        fitnesses_offspring = [ind.fitness for ind in offspring if ind.fitness.valid]
        #print(f"{gen}---Fitnesses of offsprings with valid fitnesses are {fitnesses_offspring}---")

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        #count = 1
        
        for ind, fit in zip(invalid_ind, fitnesses):
            #print(f"ind: {ind}")
            ind.fitness.values = fit[0:3]
            if fit[3].size: #If a correspondence map is present:
                corr_map_info = {"gen" : gen, "individual" : ind, "correspondence_set" : fit[3].tolist(), "fitness":ind.fitness.values[0],
                                 "inlier_rmse": ind.fitness.values[1], "mae": ind.fitness.values[2]}
                corr_map_results.append(corr_map_info)
            #print(f"--- count: {count}; gen: {gen}; Invalid offspring: {ind}, Fitness: {ind.fitness.values}---")
            #count+=1

        # Select the next generation population from parents and offspring
        pop = toolbox.select(pop + offspring, toolbox.pop_size)
        
        """
        print(f"{gen}---List of selected individuals for next generation---")
        count = 1 
        for ind in pop:
            print(f"index:{count}; ind: {ind}") 
            count +=1 
        """
        # Compile statistics about the new population
        record = mstats.compile(pop)
        logbook.record(gen=gen, evals=len(pop), **record)
        print(f"{gen} Statistics of next population: {logbook.stream}")
        
        """
         # Updating best_k_ind dictionary
        best_ind = tools.selBest(pop, best_k)
        fitnesses = [list(ind.fitness.values) for ind in best_ind]
        results = toolbox.map(toolbox.results, best_ind)
            
        mapped_ind = list(zip(best_ind, fitnesses))
        mapped_results = list(zip(best_ind,results))
        
        for ind, fitness in mapped_ind:
            print(f"individual added to dict: {ind}, individual's fitness {fitness}")
            temp = {"gen":gen , "individual": ind, "o3d_fitness" : fitness[0], 
                              "o3d_rmse" : fitness[1], "o3d_mae" : fitness[2]}
            best_k_ind.append(temp)
            
        for ind, result in mapped_results:
            temp2 = {"gen":gen , "individual": ind, "ransac_transformation": result[0], "coloredICP_transformation" : result[1], 
                     "correspondence_set" : result[2].tolist(), "fitness" : result[3],
                     "inlier_rmse" : result[4], "mae": result[5]}
            best_k_ind_results.append(temp2)
        """
        
        #print(f"Current results: {corr_map_results}")

    print(f"--End of (successful) evolution --")
    
    ## Fittest individuals from the population
    """
    best_k_ind_df = pd.DataFrame(best_k_ind)      
    print(best_k_ind_df)
    
    best_k_ind_results = pd.DataFrame(best_k_ind_results)  
    print(best_k_ind_results)
    """
    
    corr_map_results_df = pd.DataFrame(corr_map_results)
    
    ## Saving best individuals and logbook
    newpath = run_name

    if not os.path.exists(newpath):
        os.makedirs(newpath)
           
    #np.save(file = f"{newpath}/result_ransac_df.csv", arr = best_k_ind_df)
    
    """
    best_k_ind_df.to_csv(f'{newpath}/indiv_coloredICP_run{iter_}.csv', index=False) 
    best_k_ind_results.to_csv(f'{newpath}/regisresults_coloredICP_run{iter_}.csv', index=False) 
    """
    
    corr_map_results_df.to_csv(f'{newpath}/corrmapresults_coloredICP_run{iter_}.csv', index=False)
    
    results = {"gen" : logbook.select("gen"),
                "eval" : logbook.select("evals"),
                "o3d_fitness_mean" : logbook.chapters['o3d_fitness'].select("mean"),
                "o3d_fitness_max" : logbook.chapters['o3d_fitness'].select("max"),
                "o3d_rmse_mean" : logbook.chapters['o3d_rmse'].select("mean"),
                "o3d_rmse_min" : logbook.chapters['o3d_rmse'].select("min"),
                "o3d_mae_mean" : logbook.chapters['o3d_mae'].select("mean"),
                "o3d_mae_min" : logbook.chapters['o3d_mae'].select("min")}
    
    df_log = pd.DataFrame.from_dict(results) 
    df_log.to_csv(f'{newpath}/logbook_run{iter_}.csv', index=False) # Writing to a CSV file
    
merged_path_name = os.path.join(save_results_path, image_name, results_name)


In [ ]:
######################### 5) DEAP results #################################

for i in range(num_deap_runs):
    run_name = f'{merged_path_name}_{i}_{date.today()}'
    main(iter_ = i, run_name = run_name)

Initial logbook.stream    	     	o3d_fitness	   o3d_rmse  	   o3d_mae   
   	     	-----------	-------------	-------------
gen	evals	mean	max   	mean 	min  	mean 	min  
0  	30   	0   	0     	10000	10000	10000	10000
1 Statistics of next population: 1  	30   	0   	0     	10000	10000	10000	10000
2 Statistics of next population: 2  	30   	0   	0     	10000	10000	10000	10000
3 Statistics of next population: 3  	30   	0   	0     	10000	10000	10000	10000
4 Statistics of next population: 4  	30   	0   	0     	10000	10000	10000	10000
5 Statistics of next population: 5  	30   	0   	0     	10000	10000	10000	10000
6 Statistics of next population: 6  	30   	0   	0     	10000	10000	10000	10000
7 Statistics of next population: 7  	30   	0   	0     	10000	10000	10000	10000
8 Statistics of next population: 8  	30   	0   	0     	10000	10000	10000	10000
9 Statistics of next population: 9  	30   	0   	0     	10000	10000	10000	10000
10 Statistics of next population: 10 	30   	0   	0     	10000	10000	10000	

100 Statistics of next population: 100	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
101 Statistics of next population: 101	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
102 Statistics of next population: 102	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
103 Statistics of next population: 103	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
104 Statistics of next population: 104	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
105 Statistics of next population: 105	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
106 Statistics of next population: 106	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
107 Statistics of next population: 107	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
108 Statistics of next population: 108	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
109 Statistics of next population: 109	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
110 Statistics of next population: 110	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
111 Statistics of next population: 111	30   	0.002	0.07 	9666.76	

197 Statistics of next population: 197	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
198 Statistics of next population: 198	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
199 Statistics of next population: 199	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
200 Statistics of next population: 200	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
201 Statistics of next population: 201	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
202 Statistics of next population: 202	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
203 Statistics of next population: 203	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
204 Statistics of next population: 204	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
205 Statistics of next population: 205	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
206 Statistics of next population: 206	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
207 Statistics of next population: 207	30   	0.002	0.07 	9666.76	2.963	9667.04	11.1 
208 Statistics of next population: 208	30   	0.002	0.07 	9666.76	

294 Statistics of next population: 294	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
295 Statistics of next population: 295	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
296 Statistics of next population: 296	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
297 Statistics of next population: 297	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
298 Statistics of next population: 298	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
299 Statistics of next population: 299	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
300 Statistics of next population: 300	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
301 Statistics of next population: 301	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
302 Statistics of next population: 302	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
303 Statistics of next population: 303	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
304 Statistics of next population: 304	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
305 Statistics of next population: 305	30   	0.014	0.347	9333.53	

391 Statistics of next population: 391	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
392 Statistics of next population: 392	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
393 Statistics of next population: 393	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
394 Statistics of next population: 394	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
395 Statistics of next population: 395	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
396 Statistics of next population: 396	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
397 Statistics of next population: 397	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
398 Statistics of next population: 398	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
399 Statistics of next population: 399	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
400 Statistics of next population: 400	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
401 Statistics of next population: 401	30   	0.014	0.347	9333.53	2.963	9334.08	11.1 
402 Statistics of next population: 402	30   	0.014	0.347	9333.53	

...

369 Statistics of next population: 369	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
370 Statistics of next population: 370	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
371 Statistics of next population: 371	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
372 Statistics of next population: 372	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
373 Statistics of next population: 373	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
374 Statistics of next population: 374	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
375 Statistics of next population: 375	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
376 Statistics of next population: 376	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
377 Statistics of next population: 377	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
378 Statistics of next population: 378	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
379 Statistics of next population: 379	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
380 Statistics of next population: 380	30   	0.074	0.9

465 Statistics of next population: 465	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
466 Statistics of next population: 466	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
467 Statistics of next population: 467	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
468 Statistics of next population: 468	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
469 Statistics of next population: 469	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
470 Statistics of next population: 470	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
471 Statistics of next population: 471	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
472 Statistics of next population: 472	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
473 Statistics of next population: 473	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
474 Statistics of next population: 474	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
475 Statistics of next population: 475	30   	0.074	0.972	9000.44	4.074	9001.07	9.381 
476 Statistics of next population: 476	30   	0.074	0.9

62 Statistics of next population: 62 	30   	0.033	1    	9666.82	4.663	9667 	9.873
63 Statistics of next population: 63 	30   	0.033	1    	9666.82	4.663	9667 	9.873
64 Statistics of next population: 64 	30   	0.033	1    	9666.82	4.663	9667 	9.873
65 Statistics of next population: 65 	30   	0.033	1    	9666.82	4.663	9667 	9.873
66 Statistics of next population: 66 	30   	0.033	1    	9666.82	4.663	9667 	9.873
67 Statistics of next population: 67 	30   	0.033	1    	9666.82	4.663	9667 	9.873
68 Statistics of next population: 68 	30   	0.033	1    	9666.82	4.663	9667 	9.873
69 Statistics of next population: 69 	30   	0.033	1    	9666.82	4.663	9667 	9.873
70 Statistics of next population: 70 	30   	0.033	1    	9666.82	4.663	9667 	9.873
71 Statistics of next population: 71 	30   	0.033	1    	9666.82	4.663	9667 	9.873
72 Statistics of next population: 72 	30   	0.033	1    	9666.82	4.663	9667 	9.873
73 Statistics of next population: 73 	30   	0.033	1    	9666.82	4.663	9667 	9.873
74 Statistics of

162 Statistics of next population: 162	30   	0.033	1    	9666.82	4.663	9667 	9.873
163 Statistics of next population: 163	30   	0.033	1    	9666.82	4.663	9667 	9.873
164 Statistics of next population: 164	30   	0.033	1    	9666.82	4.663	9667 	9.873
165 Statistics of next population: 165	30   	0.033	1    	9666.82	4.663	9667 	9.873
166 Statistics of next population: 166	30   	0.033	1    	9666.82	4.663	9667 	9.873
167 Statistics of next population: 167	30   	0.033	1    	9666.82	4.663	9667 	9.873
168 Statistics of next population: 168	30   	0.033	1    	9666.82	4.663	9667 	9.873
169 Statistics of next population: 169	30   	0.033	1    	9666.82	4.663	9667 	9.873
170 Statistics of next population: 170	30   	0.033	1    	9666.82	4.663	9667 	9.873
171 Statistics of next population: 171	30   	0.033	1    	9666.82	4.663	9667 	9.873
172 Statistics of next population: 172	30   	0.033	1    	9666.82	4.663	9667 	9.873
173 Statistics of next population: 173	30   	0.033	1    	9666.82	4.663	9667 	9.873
174 

261 Statistics of next population: 261	30   	0.033	1    	9666.82	4.663	9667 	9.873
262 Statistics of next population: 262	30   	0.033	1    	9666.82	4.663	9667 	9.873
263 Statistics of next population: 263	30   	0.033	1    	9666.82	4.663	9667 	9.873
264 Statistics of next population: 264	30   	0.033	1    	9666.82	4.663	9667 	9.873
265 Statistics of next population: 265	30   	0.033	1    	9666.82	4.663	9667 	9.873
266 Statistics of next population: 266	30   	0.033	1    	9666.82	4.663	9667 	9.873
267 Statistics of next population: 267	30   	0.033	1    	9666.82	4.663	9667 	9.873
268 Statistics of next population: 268	30   	0.033	1    	9666.82	4.663	9667 	9.873
269 Statistics of next population: 269	30   	0.033	1    	9666.82	4.663	9667 	9.873
270 Statistics of next population: 270	30   	0.033	1    	9666.82	4.663	9667 	9.873
271 Statistics of next population: 271	30   	0.033	1    	9666.82	4.663	9667 	9.873
272 Statistics of next population: 272	30   	0.033	1    	9666.82	4.663	9667 	9.873
273 